## Подготовка

### Загрузка данных, библиотек

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

import re
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer 
nltk.download("stopwords")
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, accuracy_score

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# просмотр, где находится каталог с файлами на COLAB
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# получение доступа к каталогу и уточнение названия папок
#import os
#os.listdir('/content/drive/My Drive/Colab Notebooks/Яндекс/Проект 11 «Викишоп»/ДАННЫЕ')

['toxic_comments.csv', 'cased_L-12_H-768_A-12', 'rndlr96_EnBERT']

In [ ]:
# загрузка данных
#data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Яндекс/Проект 11 «Викишоп»/ДАННЫЕ/toxic_comments.csv')

In [ ]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
data.duplicated().sum() # подсчёт явных дубликатов

0

### Вывод

**В результате загрузки данных, установлено:**

1.	DataFrame содержит 159571 строк и 2 столбца
2.	В столбце «text» данные типа object, пропуски отсутствуют
3.	В столбце «toxic» данные типа int64, пропуски отсутствуют
4.	Явные дубликаты отсутствуют


### Предобработка данных

In [ ]:
corpus = data['text']

In [ ]:
corpus

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: text, Length: 159571, dtype: object

In [ ]:
# функция лемматизации
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemm_list = nltk.word_tokenize(text)
    lemm_text = " ".join([lemmatizer.lemmatize(l) for l in lemm_list])     
    return lemm_text

# функция очистки
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return " ".join(text.split())

In [ ]:
data['lemm_text'] = data['text'].progress_apply(clear_text)
data['lemm_text'] = data['lemm_text'].progress_apply(lemmatize)

100%|██████████| 159571/159571 [02:05<00:00, 1270.48it/s]


In [ ]:
data

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,And for the second time of asking when your vi...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That is a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm there no actual article for prosti...
159569,And it looks like it was actually you who put ...,0,And it look like it wa actually you who put on...


In [ ]:
data_lemm = data

In [ ]:
features_start = data_lemm['lemm_text']
target_start = data_lemm['toxic']

In [ ]:
# исследование баланса классов
features_zeros = features_start[target_start == 0] 
features_ones = features_start[target_start == 1] 

target_zeros  = target_start[target_start == 0] 
target_ones = target_start[target_start == 1] 

print(features_zeros.shape)
print(features_ones.shape)
print(target_zeros.shape)
print(target_ones.shape)

(143346,)
(16225,)
(143346,)
(16225,)


In [ ]:
print('Баланс классов = ', round(target_ones.shape[0] / target_zeros.shape[0] * 10), ':',
      round((round(100 - target_ones.shape[0] / target_zeros.shape[0] * 100) / 10)))

Баланс классов =  1 : 9


In [ ]:
features_train_start, features_test_start, target_train, target_test = train_test_split(features_start, target_start, test_size=0.25)

In [ ]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train_start)
features_test = count_tf_idf.transform(features_test_start)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Вывод

**В результате предобработки данных:**

1.	Выполнена лемматизация и очистка данных
2.	Исследован баланс классов (1:9)
3.	Выполнено кодирование и векторизация целевых признаков
4.	Выборка разделена на тренировочную и тестовую (75:25)


## Обучение

###  LogisticRegression

In [ ]:
model = LogisticRegression()
parametrs = { 'C': range(1, 20, 2),
               'class_weight':['balanced', None] }
search = GridSearchCV(model, parametrs, cv=3, scoring='f1')
search.fit(features_train, target_train)
best_model_LogisticRegression = search.best_estimator_
print('# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #')
print('Модель:', best_model_LogisticRegression)
print('Параметры лучшей модели:', search.best_params_)
print('F1:', round(search.best_score_, 3))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
Модель: LogisticRegression(C=11, class_weight='balanced')
Параметры лучшей модели: {'C': 11, 'class_weight': 'balanced'}
F1: 0.762


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


### CatBoostClassifier

In [ ]:
model = CatBoostClassifier()
parametrs = { 'depth': range (1, 2, 1),
              'n_estimators': range (1, 2, 1) }
search = GridSearchCV(model, parametrs, cv=3, scoring='f1')
search.fit(features_train, target_train)
best_model_CatBoostClassifier = search.best_estimator_
print('# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #')
print('Модель:', best_model_CatBoostClassifier)
print('Параметры лучшей модели:', search.best_params_)
print('best_score:', round(search.best_score_, 3))

Learning rate set to 0.5
0:	learn: 0.3709003	total: 286ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3714175	total: 286ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3681059	total: 289ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3702821	total: 434ms	remaining: 0us
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
Модель: <catboost.core.CatBoostClassifier object at 0x7faabe99f090>
Параметры лучшей модели: {'depth': 1, 'n_estimators': 1}
best_score: 0.264


### RandomForestClassifier

In [ ]:
model = RandomForestClassifier()
parametrs = { 'max_depth': range (1, 4),
              'n_estimators': range (1, 4) }
search = GridSearchCV(model, parametrs, cv=3, scoring='f1')
search.fit(features_train, target_train)
best_model_RandomForestClassifier = search.best_estimator_
print('# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #')
print('Модель:', best_model_RandomForestClassifier)
print('Параметры лучшей модели:', search.best_params_)
print('best_score:', round(search.best_score_, 3))

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
Модель: RandomForestClassifier(max_depth=2, n_estimators=2)
Параметры лучшей модели: {'max_depth': 2, 'n_estimators': 2}
best_score: 0.059


### LGBMClassifier

In [ ]:
model = LGBMClassifier()
parametrs = { 'max_depth': range (1, 4),
              'n_estimators': range (1, 4) }
search = GridSearchCV(model, parametrs, cv=3, scoring='f1')
search.fit(features_train, target_train)
best_model_LGBMClassifier = search.best_estimator_
print('# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #')
print('Модель:', best_model_LGBMClassifier)
print('Параметры лучшей модели:', search.best_params_)
print('best_score:', round(search.best_score_, 3))

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
Модель: LGBMClassifier(max_depth=3, n_estimators=3)
Параметры лучшей модели: {'max_depth': 3, 'n_estimators': 3}
best_score: 0.001


### Вывод

**В результате обучения и подбора лучших параметров моделей, установлено:**

1.  Модель: LogisticRegression(C=11, class_weight='balanced')
    * Параметры лучшей модели: {'C': 11, 'class_weight': 'balanced'}
    * F1: 0.762
2.  Модель: <catboost.core.CatBoostClassifier object at 0x7faabe99f090>
    * Параметры лучшей модели: {'depth': 1, 'n_estimators': 1}
    * F1: 0.264
3.  Модель: RandomForestClassifier(max_depth=2, n_estimators=2)
    * Параметры лучшей модели: {'max_depth': 2, 'n_estimators': 2}
    * F1: 0.059
4.  Модель: LGBMClassifier(max_depth=3, n_estimators=3)
    * Параметры лучшей модели: {'max_depth': 3, 'n_estimators': 3}
    * F1: 0.001

## Проверка качества модели на тестовой выборке

In [ ]:
list_model = [best_model_LogisticRegression,
              best_model_CatBoostClassifier,
              best_model_RandomForestClassifier,
              best_model_LGBMClassifier]

In [ ]:
list_model

[LogisticRegression(C=11, class_weight='balanced'),
 RandomForestClassifier(max_depth=2, n_estimators=2),
 LGBMClassifier(max_depth=3, n_estimators=3)]

In [ ]:
best_model = None
best_result = 0
for mod in list_model:
  mod.fit(features_train, target_train)
  test_predictions = mod.predict(features_test)
  result = f1_score(target_test, test_predictions)
  if best_result < result:
    best_result = result
    best_model = mod
print('# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #')
print("Лучшая модель:", best_model)
print("F1 наилучшей модели на тестовой выборке:", round(best_result, 3))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Learning rate set to 0.5
0:	learn: 0.3702821	total: 454ms	remaining: 0us
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
Лучшая модель: LogisticRegression(C=11, class_weight='balanced')
F1 наилучшей модели на тестовой выборке: 0.761


### Вывод

**В результате проверки качества модели на тестовой выборке:**

1.  Лучшая модель: LogisticRegression(C=11, class_weight='balanced')
2.  F1 наилучшей модели на тестовой выборке: 0.761
3.  Значение F1 удовлетворяет условию: F1 не меньше 0.75.
4.  Попытаемся улучшить качество предсказания других (кроме LogisticRegression()) моделей за счёт балансировки классов.

### Борьба с дисбалансом

*Так как баланс примерно 1 к 9 попытаемся сначала уменьшить больший класс и посмотреь на значение F1, а затем увеличить меньший класс.*

In [ ]:
exp_list_model = [best_model_CatBoostClassifier,
              best_model_RandomForestClassifier,
              best_model_LGBMClassifier]

In [ ]:
# функция уменьшения выборки
def downsample(features_train_s, target_t, fraction):

  target_zeros = target_t[target_t == 0]
  target_ones = target_t[target_t == 1]

  target_zeros_downsample = target_zeros.sample(round(target_zeros.shape[0] * fraction), random_state=12345)
  target_downsample = pd.concat([target_zeros_downsample, target_ones])
  features_downsample = features_train_s.iloc[target_downsample.index]
  features_downsample, target_downsample = shuffle(features_downsample, target_downsample, random_state=12345)
  
  return features_downsample, target_downsample

In [ ]:
  features_train_start_exp = features_train_start.reset_index(drop=True)
  target_train_exp = target_train.reset_index(drop=True)

In [ ]:
target_train_exp.shape

(119678,)

In [ ]:
features_train_start_exp.shape

(119678,)

In [ ]:
# поиск наиболее оптимального значения МНОЖИТЕЛЯ (хN), уменьшающего выборку, а также лучшей модели и её F1
for j in  np.arange(0.1, 0.9, 0.1):
  features_downsampled, target_downsampled = downsample(features_train_start_exp, target_train_exp, j)

  features_downsampled = count_tf_idf.fit_transform(features_downsampled)
  features_test = count_tf_idf.transform(features_test_start)

  best_model = None
  best_result = 0
  best_хN = 0.1
  for mod in exp_list_model:
    mod.fit(features_downsampled, target_downsampled)
    test_predictions = mod.predict(features_test)
    result = f1_score(target_test, test_predictions)
    if best_result < result:
      best_result = result
      best_model = mod
      best_хN = j
print('# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #')
print("Лучшая модель:", best_model)
print("F1 наилучшей модели на тестовой выборке:", round(best_result, 3))
print("Лучший МНОЖИТЕЛЬ (хN), уменьшающий выборку:", best_хN)

Learning rate set to 0.5
0:	learn: 0.6408057	total: 68.2ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.6133017	total: 103ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.5653127	total: 143ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.5221810	total: 179ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.4878598	total: 219ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.4551786	total: 257ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.4291228	total: 297ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.4081993	total: 337ms	remaining: 0us
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
Лучшая модель: <catboost.core.CatBoostClassifier object at 0x7faabe99f090>
F1 наилучшей модели на тестовой выборке: 0.256
Лучший МНОЖИТЕЛЬ (хN), уменьшающий выборку: 0.8


In [ ]:
# функция увеличение выборки
def upsample(features_train_s, target_t, repeat):
    features_zeros = features_train_s[target_t == 0]
    features_ones = features_train_s[target_t == 1]
    target_zeros = target_t[target_t == 0]
    target_ones = target_t[target_t == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [ ]:
# поиск наиболее оптимального значения МНОЖИТЕЛЯ (хN), увеличивающего выборку, а также лучшей модели и её F1
for j in  np.arange(1, 7, 2):
  features_upsampled, target_upsampled = upsample(features_train_start_exp, target_train_exp, j)

  features_upsampled = count_tf_idf.fit_transform(features_upsampled)
  features_test = count_tf_idf.transform(features_test_start)

  best_model = None
  best_result = 0
  best_хN = 1
  for mod in exp_list_model:
    mod.fit(features_upsampled, target_upsampled)
    test_predictions = mod.predict(features_test)
    result = f1_score(target_test, test_predictions)
    if best_result < result:
      best_result = result
      best_model = mod
      best_хN = j
print('# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #')
print("Лучшая модель:", best_model)
print("F1 наилучшей модели на тестовой выборке:", round(best_result, 3))
print("Лучший МНОЖИТЕЛЬ (хN), увеличивающий выборку:", best_хN)

Learning rate set to 0.5
0:	learn: 0.3702820	total: 447ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.5506162	total: 497ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.6136533	total: 574ms	remaining: 0us
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
Лучшая модель: LGBMClassifier(max_depth=3, n_estimators=3)
F1 наилучшей модели на тестовой выборке: 0.373
Лучший МНОЖИТЕЛЬ (хN), увеличивающий выборку: 5


### Вывод

**В результате борьбы с дисбалансом и повторного подбора лучших параметров моделей (кроме LogisticRegression()), установлено:**

1.  Лучшая модель: LGBMClassifier(max_depth=3, n_estimators=3)
2.  F1 наилучшей модели на тестовой выборке: 0.373
3.  Лучший МНОЖИТЕЛЬ (хN), увеличивающий выборку: 5
4.  Прирост F1 порядка 11%, однако не удовлетворяет условию: F1 не меньше 0.75.


## Общий выводы

**В результате загрузки данных, установлено:**

1.  DataFrame содержит 159571 строк и 2 столбца
2.  В столбце «text» данные типа object, пропуски отсутствуют
3.  В столбце «toxic» данные типа int64, пропуски отсутствуют
4.  Явные дубликаты отсутствуют.

**В результате предобработки данных:**

1.  Выполнена лемматизация и очистка данных
2.  Исследован баланс классов (1:9)
3.  Выполнено кодирование и векторизация целевых признаков
4.  Выборка разделена на тренировочную и тестовую (75:25)

**В результате обучения и подбора лучших параметров моделей, установлено:**

1.  Модель: LogisticRegression(C=11, class_weight='balanced')
    * Параметры лучшей модели: {'C': 11, 'class_weight': 'balanced'}
    * F1: 0.762
2.  Модель: <catboost.core.CatBoostClassifier object at 0x7faabe99f090>
    * Параметры лучшей модели: {'depth': 1, 'n_estimators': 1}
    * F1: 0.264
3.  Модель: RandomForestClassifier(max_depth=2, n_estimators=2)
    * Параметры лучшей модели: {'max_depth': 2, 'n_estimators': 2}
    * F1: 0.059
4.  Модель: LGBMClassifier(max_depth=3, n_estimators=3)
    * Параметры лучшей модели: {'max_depth': 3, 'n_estimators': 3}
    * F1: 0.001

**В результате проверки качества модели на тестовой выборке:**

1.  Лучшая модель: LogisticRegression(C=11, class_weight='balanced')
2.  F1 наилучшей модели на тестовой выборке: 0.761
3.  Значение F1 удовлетворяет условию: F1 не меньше 0.75.
4.  Попытаемся улучшить качество предсказания других (кроме LogisticRegression()) моделей за счёт балансировки классов.

**В результате борьбы с дисбалансом и повторного подбора лучших параметров моделей (кроме LogisticRegression()), установлено:**

1.  Лучшая модель: LGBMClassifier(max_depth=3, n_estimators=3)
2.  F1 наилучшей модели на тестовой выборке: 0.373
3.  Лучший МНОЖИТЕЛЬ (хN), увеличивающий выборку: 5
4.  Прирост F1 порядка 11%, однако не удовлетворяет условию: F1 не меньше 0.75.

**РЕЗЮМЕ:**

1.  Лучшая модель: LogisticRegression(C=11, class_weight='balanced')
2.  F1 модели на тестовой выборке: 0.761